In [22]:
from common import *
import operator

In [23]:
_TEST_ = False

In [24]:
if __name__=='__main__':
    _TEST_ = True
    interactive(rows=2000, cols=2000)

# -=#=-=#=-=#=-=#=-=#=- Features selection logic -=#=-=#=-=#=-=#=-=#=-=#=-

### -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- Abstract Function datatype -=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-

In [25]:
from types import FunctionType

##### item checker

In [26]:
def item_check(a):
    if not isinstance(a, tuple):
        a = (a,)
    return a

#### Function super useful datatype

In [27]:
class Function(type):
    def __new__(self, *args, **kw):
        noattach = '_noattach_' in kw
        if len(args) == 1:
            if isinstance(args[0], __class__):
                return args[0]
            assert callable(args[0])
            if 'name' in kw:
                name = kw['name']
            else:
                name = args[0].__name__
            parents = ()
            attrs = {'__call__': args[0]}
            noattach = True
        else:
            name, parents, attrs = args
        for k, v in list(dict.items(attrs)):
            if k == '__getitem__':
                assert isinstance(v, FunctionType), "just for easier work on our side. it is a function(self, index)"
                if isinstance(v(self, 0), FunctionType): # if getitem returns a function
                    old_v = v
                    v = lambda self, a: __class__(old_v(self, item_check(a)))
            if not isinstance(v, classmethod) and callable(v) and not isinstance(v, __class__) and not noattach:
                attrs[k] = classmethod(v)
            
        return super().__new__(self, name, parents, attrs)
    
    def __init__(self, *args, **kw):
        #SEE('init', args, kw)
        if len(args) == 1:
            return
        name, parents, attrs = args
        for k in attrs:
            v = getattr(self, k)
            #SEE('init ->', k, v)
            if k[:1] != '_' and callable(v):
                setattr(self, k, __class__(v, _noattach_=True, name=k))
        
    def __call__(self, *a, **k):
        assert '__call__' in self.__dict__, f'__call__ not defined for Function object with name "{self.__name__}"'
        return self.__call__(*a, **k)
        
    def __getitem__(self, *a, **k):
        assert '__getitem__' in self.__dict__, f'__getitem__ not defined for Function object with name "{self.__name__}"'
        return self.__getitem__(*a, **k)
    
    __invert__ = lambda f: __class__(lambda *a, **k: ~f(*a,**k))
    __and__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) & g(*a,**k))
    __or__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) | g(*a,**k))
    __xor__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) ^ g(*a,**k))

In [28]:
# testing
if _TEST_:
    class What(metaclass=Function):
        def __call__(self, x, y):
            return np.sqrt(x**2 + y**2)
        def hi(self, *s):
            pass
            #SEE('hi, ' + s[0])
        __getitem__ = lambda self, a: lambda arg: (SEE('this is getitem', arg, arg*a), 'answer')[1]
    What.hi('u')
    What.hi#.__call__
    /SEE What[5].__class__
    /SEE What[5](6)

##### item getter

In [29]:
class Itemgetter():
    def __init__(self, func):
        self._func = Function(func)
    def __getitem__(self, a):
        return self._func(item_check(a))

In [30]:
# testing
if _TEST_:
    guy = Itemgetter(lambda *a: SEE(*a))
    guy[5.7,8]

In [31]:
# testing yo!
if _TEST_:
    class Yo():
        def __call__(self, x):
            SEE(x, '!')
    yo = Yo()
    yo.__call__ = lambda x: SEE('fuck', x)
    yo(5)

#### Function query language implementation

In [32]:
def scan(s):
    s = s.replace(' ','').replace('\n','').replace('\t','').replace('\r','') + '\0'
    i0, i = 0, 0
    while i0 < len(s)-1 and i < len(s):
        c = s[i]
        if 'a' <= c <= 'z' or 'A' <= c <= 'Z' or '0' <= c <= '9' or c == '_':
            i += 1
            continue
        elif c == '\0':
            yield s[i0:i]
            break
        elif c in '.~&|^()':
            if i0 < i:
                yield s[i0:i]
            yield c
            i0 = i = i+1
        elif c in '[{<':
            if i0 < i:
                yield s[i0:i]
            i0 = i
            op = c
            cl = {'[':']','{':'}','<':'>'}[op]
            opened = 1
            while opened > 0:
                i += 1
                opened += 1 if s[i] == op else -1 if s[i] == cl else 0
            i += 1
            if op != '<': # if this is not a comment
                yield s[i0:i]
            i0 = i
        else:
            raise AssertionError('bad syntax in input to scanner')

            
Token = namedtuple('Token', 'type value')

def tokenize(s):
    prevIsField = False
    for w in s:
        a = w[0]
        if w == '.':
            yield Token('self', '')
        elif '0' <= a <= '9':
            yield Token('literal', w)
        elif 'a' <= a <= 'z' or 'A' <= a <= 'Z' or a in '_':
            yield Token('field', w)
            prevIsField = True
            continue
        elif w in ['&','^','|']:
            yield Token('op2', w)
        elif w in ['~']:
            yield Token('op1', w)
        elif w in ['(']:
            yield Token('_group', w)
        elif w in [')']:
            yield Token('group_', w)
        elif a == '{':
            assert prevIsField, '{} section must have been preceded by a field name'
            yield Token('op2', 'query')
            yield Token('raw', w)
        elif a == '[':
            if not prevIsField:
                yield Token('self', '')
            yield Token('op2', 'getitem')
            yield Token('raw', w)
        else:
            assert False, f'invalid word encountered in tokenizer input: {w}'
        prevIsField = False
        
class TokenInfo(O()):
    self = 'self, the owning Function instance'
    field = 'field name of Function instance'
    op2 = 'binary operator (look in the map)'
    op1 = 'unary operator (look in the map)'
    _group = "open grouping (just '(')"
    group_ = "close grouping (just ')')"
    raw = "raw data to be fed to operator"
    literal = "literal value. either '0' or '1'"


def normalize(s):
    def reduce_op2(s):
        '''removes repeated binary operators or binary operators missing left/right argument'''
        prev = None
        hardprev = None
        for t in s:
            if (prev is not None and prev.type=='_group' or hardprev is None) and t.type=='op2':
                #^ e.g. "(| A | B)" or "& B", remove useless first binop
                pass # don't set prev=t and defer yielding prev (
            elif hardprev is not None and hardprev.type=='op2' and t.type=='op2': # repeat op2
                assert hardprev.value==t.value, 'repeated op2 must be the same'
                # don't set prev=t, no non-redunt piece of resulting query has been seen
            elif prev is not None and prev.type=='op2' and t.type=='group_':
                # don't yield prev
                prev = t
            elif prev is None:
                # attempt to yield `prev` next loop iter
                prev = t
            else:
                yield prev
                prev = t
            hardprev = t
        if prev is not None and prev.type!='op2':
            yield prev
    def reduce_empty(s):
        '''replace empty groups () with 0'''
        prev = None
        for t in s:
            if prev is None:
                prev = t
            elif prev.type=='_group' and t.type=='group_':
                prev = Token('literal', '0')
            else:
                yield prev
                prev = t
        yield prev
    yield from reduce_empty(reduce_op2(s))
    
def parse(s):
    stack = []
    for t in s:
        if t.type in ['field', 'self', 'raw', 'literal']:
            yield t
        elif t.type in ['op2', 'op1']:
            while stack and stack[-1].type in ['op2', 'op1'] and parse.order[t.value] >= parse.order[stack[-1].value]:
                yield stack.pop()
            stack.append(t)
        elif t.type == '_group':
            stack.append(t)
        elif t.type == 'group_':
            while True:
                if not stack:
                    raise AssertionError('too many close parentheses')
                x = stack.pop()
                if x.type == '_group':
                    break
                else:
                    assert x.type in ['op2', 'op1'], 'internal error'
                    yield x
    while stack:
        x = stack.pop()
        if x.type == '_group':
            raise AssertionError('too many open parentheses')
        assert x.type in ['op2', 'op1'], 'internal error'
        yield x
parse.order = {'query': 0, 'getitem': 0, '~': 1, '&': 2, '^': 3, '|': 4}

    
def query(self, qs):
    revpol = list(parse(normalize(tokenize(scan(qs)))))
    stack = []
    BUG.stack = stack
    for t in revpol:
        if t.type == 'self':
            stack.append(self)
        elif t.type == 'field':
            stack.append(getattr(self, t.value))
        elif t.type == 'literal':
            if t.value == '0':
                stack.append(Function(lambda*a,**k:False))
            elif t.value == '1':
                stack.append(Function(lambda*a,**k:True))
            else:
                assert False, f'unsupported literal value found, token={t}'
        elif t.type == 'raw':
            stack.append(t.value)
        elif t.type == 'op1':
            a = stack.pop()
            x = query.opTbl[t.value](a)
            stack.append(x)
        elif t.type == 'op2':
            b, a = stack.pop(), stack.pop()
            x = query.opTbl[t.value](a, b)
            stack.append(x)
    return stack.pop()

class GetItemHelper(metaclass=Singleton):
    def __getitem__(self, a):
        return a

query.opTbl = {
    '~': operator.__invert__,
    '&': operator.__and__,
    '^': operator.__xor__,
    '|': operator.__or__,
    'query': lambda f, q: f.query(q[1:-1]),
    'getitem': lambda f, a: f[eval(f'GetItemHelper()[{a[1:-1]}]')]
}

Function.query = query

In [33]:
# testing function query
if _TEST_:
    qs = 'what2{f_ck} & no_[5,62,6:6] <whatevercomment> |  [9] & ~(ho_10|[3,5:9])'
    scanned = list(scan(qs))
    assert (scanned == ['what2','{f_ck}','&','no_','[5,62,6:6]','|','[9]','&','~','(','ho_10','|','[3,5:9]',')']), (
            'Unit test fail for scanner')    
    tokens = list(tokenize(scanned)) #TODO make unit test
    parsed = list(parse(tokens)) #TODO make unit test
    SEE(qs)
    SEE('-'*99)
    SEE(scanned)
    SEE('-'*99)
    for line in tokens:
        SEE(line)
    SEE('-'*99)
    for line in parsed:
        SEE(line)

In [34]:
def stringify(self, name=None):
    name = self.__name__ if name is None else name
    cond = lambda field: not(field=='query' or field[:1]=='_' and field!='__getitem__')
    if name == '__getitem__':
        yield '[]'
        return
    yield name
    if not any(cond(f) for f in dir(self)):
        return
    yield '{'
    for field in self.__dict__:
        if not cond(field):
            continue
        value = getattr(self, field)
        yield from stringify(getattr(self, field), field)
        yield ','
    yield '}'
Function.stringify = stringify